In [ ]:
#reading 2012 comb census data
path_comb_2012=os.path.join('output_census', 'census_comb_2012.csv')
census_2012=pd.read_csv(path_comb_2012)

In [ ]:
#plotting
fig, ax=plt.subplots(figsize=(15,15))
census_2012.plot(kind='scatter', x='Lng', y='Lat', alpha=0.2, 
                s=census_2012['Population']/100,label='Population',
                 c='House Value', cmap=plt.get_cmap('jet'),
                colorbar=True, ax=ax)
#ax.set_xlimit([-175,-125])
plt.show()

In [ ]:
#plotting
fig, ax=plt.subplots(figsize=(15,15))
census_ca.plot(kind='scatter', x='Lng', y='Lat', alpha=0.2, 
                s=census_ca['Population']/100,label='Population',
                 c='House Value', cmap=plt.get_cmap('jet'),
                colorbar=True, ax=ax)
#ax.set_ylimit([-125,-175])
plt.show()

In [ ]:
#plotting
fig, ax=plt.subplots(figsize=(15,15))
census_ca.plot(kind='scatter', x='Lng', y='Lat', alpha=0.2, 
                s=census_ca['Population']/100,label='Population',
                 c='Household Income', cmap=plt.get_cmap('jet'),
                colorbar=True, ax=ax)
#ax.set_ylimit([-125,-175])
plt.show()

In [ ]:
#plotting all correlation using seaborn heatmap
corr=census_ca.corr()
import seaborn as sns
fig, ax=plt.subplots(figsize=(15,15))
sns.heatmap(corr,vmin=-1, vmax=1, annot=True, ax=ax, cmap='BrBG')
plt.show()

In [ ]:
attrb=['Population', 'Median Age', 'Household Income',
       'Per Capita Income', 'Poverty Rate', 'Unemployment Rate', 'House Value',
       'House Construction Year', 'Monthly Owner Cost', 'Monthly Rent']
sns.pairplot(census_ca[attrb], diag_kind="hist")
plt.show()